In [1]:
import pandas as pd
from ethtx import EthTx, EthTxConfig
from ethtx.models.decoded_model import DecodedTransaction
from typing import List

In [ ]:
txhash = '0x4c13a74b90bd9b6da0a648beebf355010b959ca714b3317461c0017b713fe354'

ethtx_config = EthTxConfig(
    mongo_connection_string="mongomock://localhost/ethtx",  ##MongoDB connection string,
    etherscan_api_key="VQXUYMB6T6I2E9SF67JPKP6YCYFW5P645A",  ##Etherscan API key,
    web3nodes={
        "mainnet": {
            "hook": "https://api.archivenode.io/58db800c-c787-41c1-96d5-ed35e126eb35",  # multiple nodes supported, separate them with comma
            "poa": True
        }
    },
    default_chain="mainnet",
    etherscan_urls={"mainnet": "https://api.etherscan.io/api", },
)

ethtx = EthTx.initialize(ethtx_config)
""" decoded_transaction: DecodedTransaction = ethtx.decoders.decode_transaction(
    '0x1594180567438ad258c98e263b412f55f62a23ebb5e9056089e9f11a513d4816')

print(decoded_transaction) """

In [ ]:
from ethtx.models.w3_model import W3Transaction, W3Block, W3Receipt, W3CallTree
web3provider = ethtx.providers.web3provider


# read raw transaction data directly from the node
w3transaction: W3Transaction = web3provider.get_transaction(txhash)
w3block: W3Block = web3provider.get_block(w3transaction.blockNumber)
w3receipt: W3Receipt = web3provider.get_receipt(w3transaction.hash.hex())
w3calls: W3CallTree = web3provider.get_calls(w3transaction.hash.hex())

print(w3transaction.dict())


In [ ]:
from ethtx.models.decoded_model import (
    DecodedTransfer,
    DecodedBalance,
    DecodedEvent, DecodedCall,
)
from ethtx.models.objects_model import Transaction, Event, Block, Call

# read the raw transaction from the node
transaction = Transaction.from_raw(
    w3transaction=w3transaction, w3receipt=w3receipt, w3calltree=w3calls
)

# get proxies used in the transaction
proxies = ethtx.decoders.get_proxies(transaction.root_call, "mainnet")

block: Block = Block.from_raw(
    w3block=web3provider.get_block(transaction.metadata.block_number),
    chain_id="mainnet",
)

# decode transaction components
abi_decoded_events: List[Event] = ethtx.decoders.abi_decoder.decode_events(
    transaction.events, block.metadata, transaction.metadata
)
abi_decoded_calls: DecodedCall = ethtx.decoders.abi_decoder.decode_calls(
    transaction.root_call, block.metadata, transaction.metadata, proxies
)
abi_decoded_transfers: List[
    DecodedTransfer
] = ethtx.decoders.abi_decoder.decode_transfers(abi_decoded_calls, abi_decoded_events)
abi_decoded_balances: List[DecodedBalance] = ethtx.decoders.abi_decoder.decode_balances(
    abi_decoded_transfers
)


In [ ]:

# decode a single event
raw_event: Event = transaction.events[0]
abi_decoded_event: DecodedEvent = ethtx.decoders.abi_decoder.decode_event(
    raw_event, block.metadata, transaction.metadata
)

# decode a single call
raw_call: Call = transaction.root_call.subcalls[0]
abi_decoded_call: DecodedCall = ethtx.decoders.abi_decoder.decode_call(
    raw_call, block.metadata, transaction.metadata, proxies
)


In [ ]:
from ethtx.models.decoded_model import DecodedTransactionMetadata

# semantically decode transaction components
decoded_metadata: DecodedTransactionMetadata = (
    ethtx.decoders.semantic_decoder.decode_metadata(
        block.metadata, transaction.metadata, "mainnet"
    )
)
decoded_events: List[DecodedEvent] = ethtx.decoders.semantic_decoder.decode_events(
    abi_decoded_events, decoded_metadata, proxies
)

decoded_calls: Call = ethtx.decoders.semantic_decoder.decode_calls(
    abi_decoded_calls, decoded_metadata, proxies
)
decoded_transfers: List[
    DecodedTransfer
] = ethtx.decoders.semantic_decoder.decode_transfers(
    abi_decoded_transfers, decoded_metadata
)
decoded_balances: List[
    DecodedBalance
] = ethtx.decoders.semantic_decoder.decode_balances(
    abi_decoded_balances, decoded_metadata
)


In [ ]:

""" # semantically decode a single event
decoded_event: DecodedEvent = ethtx.decoders.semantic_decoder.decode_event(
    abi_decoded_events[0], decoded_metadata, proxies
)
# semantically decode a single call
decoded_call: Call = ethtx.decoders.semantic_decoder.decode_call(
    abi_decoded_calls.subcalls[0], decoded_metadata, proxies
) """


In [ ]:
import sys
import json

sys.path.append("../src/")

from com.cryptobot.utils.logger import PrettyLogger

transaction_dict = transaction.dict()
abi_decoded_events_dict = [ev.dict() for ev in abi_decoded_events]
abi_decoded_calls_dict = decoded_calls.dict()
abi_decoded_transfers_dict = [t.dict() for t in abi_decoded_transfers]
abi_decoded_balances_dict = [b.dict() for b in abi_decoded_balances]

with open(f'../data/{txhash}_transaction.json', mode='a') as fp:
    json.dump(transaction_dict, fp, default=str)
    fp.close()

with open(f'../data/{txhash}_events.json', mode='a') as fp:
    json.dump(abi_decoded_events_dict, fp, default=str)
    fp.close()

with open(f'../data/{txhash}_calls.json', mode='a') as fp:
    json.dump(abi_decoded_calls_dict, fp, default=str)
    fp.close()

with open(f'../data/{txhash}_transfers.json', mode='a') as fp:
    json.dump(abi_decoded_transfers_dict, fp, default=str)
    fp.close()

with open(f'../data/{txhash}_balances.json', mode='a') as fp:
    json.dump(abi_decoded_balances_dict, fp, default=str)
    fp.close()


In [34]:
import sys
import json

sys.path.append("../src/")

from com.cryptobot.utils.path import get_data_path

tokens = pd.read_csv(get_data_path() + 'tokens.csv')
tokens_holders_df = pd.read_csv(get_data_path() + 'tokens_holders.csv')

# sort them by symbol
tokens.sort_values(by=['symbol'], inplace=True)
tokens_holders_df.sort_values(by=['token_symbol'], inplace=True)

whale = tokens_holders_df.copy()[tokens_holders_df['address']
                                 == '0xcb018790c03bdfd33a82354dee4e320f1398d66f']

del whale['percentage']
del tokens['address']

whale_tokens = tokens.copy()[tokens['symbol'].isin(list(whale['token_symbol']))]
whale.reset_index(drop=True, inplace=True)
whale_tokens.reset_index(drop=True, inplace=True)

whale.loc[:, ('wallet_holdings_usd')] = whale.qty.mul(whale_tokens.price_usd)
total_usd = whale.wallet_holdings_usd.sum()
whale.loc[:, ('wallet_portfolio_alloc')] = (whale.wallet_holdings_usd.mul(100))/total_usd

whale = whale.merge(tokens, left_on='token_symbol', right_on='symbol')
whale.loc[:, ('wallet_market_percent')] = (
    whale.wallet_holdings_usd.mul(100))/whale.market_cap
whale.rename(columns={'address_x': 'wallet_address', 'address_y': 'token_address',
             'name': 'token_name', 'price_usd': 'token_price_usd', 'qty': 'wallet_holdings_qty', 'market_cap': 'token_market_cap'}, inplace=True)
whale.sort_values(by=['wallet_portfolio_alloc', 'wallet_market_percent'],
                  ascending=False, inplace=True)

del whale['symbol']

whale


,rank,address,wallet_holdings_qty,token_address,token_symbol,wallet_holdings_usd,wallet_portfolio_alloc,token_name,token_market_cap,token_price_usd,wallet_market_percent
0,138,0xcb018790c03bdfd33a82354dee4e320f1398d66f,5.267425e-09,0x85f138bfee4ef8e540890cfb48f620571d67eda3,AVAX,8.059161e-08,100.0,AVALANCHE,4.531408e+09,15.3,1.778511e-15
